In [ ]:
import gym
import keras
import numpy as np

In [ ]:
def prepro(I):
    """ prepro 210x160x3 uint8 frame into 6400 (80x80) 1D float vector """
    I = I[35:195] # crop
    I = I[::2,::2,0] # downsample by factor of 2
    I[I == 144] = 0 # erase background (background type 1)
    I[I == 109] = 0 # erase background (background type 2)
    I[I != 0] = 1 # everything else (paddles, ball) just set to 1
    return I.astype(np.float32)

def run_episode(env,agent, n_episodes=1, ep = 0.05):
    '''
    A pure run_episode with an agent. Does nothing else.
    Outputs 3 lists with one entry for every step: state, action and reward
    '''
    state, action, reward, value = [], [], [], []
    episode_number = 0
    observation = env.reset()
    prev_x = None # used in computing the difference frame

    while True:
        # Process observation
        cur_x = prepro(observation)
        x = cur_x - prev_x if prev_x is not None else np.zeros(80*80)
        x = x.reshape((1, -1))
        prev_x = cur_x

        # Execute a step:
        v, a = agent(x,ep=ep)
        observation, r, done, info = env.step(a)

        # Store variables from run
        state.append(x) # note that this is the state before the action was taken
        action.append(a)
        reward.append(r)
        value.append(v)

        if done:
            episode_number += 1
            observation = env.reset() # reset env
            prev_x = None

            if episode_number >= n_episodes:
                
                reward = np.vstack(reward)
                action = np.vstack(action)
                state = np.vstack(state)
                value = np.vstack(value)
                return state,action,reward, value


# Model

In [ ]:
from keras.models import Sequential, Input, Model
from keras.layers import Dense, Activation

inp = Input(shape=(80*80,))
h = Dense(200, activation='relu')(inp)
#h = Dense(500, activation='relu')(h)
#h = Dense(200, activation='relu')(h)
V = Dense(1,activation="tanh", name = "critic")(h)
pi = Dense(6,activation='softmax',name="policy-output")(h)

model = Model(inputs=inp, 
                  outputs=[V,pi])

import keras.backend as K
from keras.losses import mean_squared_error

def weighted_crossentropy(y_true,y_pred):
    reward_true = y_true[:,0]
    action_true = y_true[:,1:]
    ce = K.categorical_crossentropy(y_pred, action_true)
    wce = ce*reward_true
    print ce
    print wce
    return wce

model.compile(loss=[mean_squared_error, weighted_crossentropy],
               optimizer=keras.optimizers.RMSprop(lr=0.005, rho=0.9, epsilon=1e-08, decay=0.0))

In [ ]:
def agent(state, ep = 0.05):
    V, aprob = model.predict(state)
    
    if np.random.uniform() < ep:
        action = env.action_space.sample()
    else:
        action = np.argmax(aprob)
    return V,action

## Play

In [ ]:
from keras.utils import to_categorical
#from keras.callbacks import 
env = gym.make("Pong-v0")
running_reward = -20.5
batch = 0
batch_ep = 5 #how many episodes to run per batch
ep = 0.1 # probability of random action
while True:
    state,action,reward, value = run_episode(env, agent, n_episodes=batch_ep, ep = ep)

    # Update model
    Vt = np.roll(value,-1) # take lead of Vt
    R = reward + (reward==0.0)*Vt*0.99 # Value is reward if done (same as reward!=0), else it is predicted value else
    weight = (R-Vt)
    pi_true_weights = np.hstack([weight, to_categorical(action)]) # Format truth value to be in accordance to weighted_crossentropy
    h = model.fit(state, [R, pi_true_weights], verbose=0)
    vloss = h.history.items()[2][1][0]
    piloss = h.history.items()[1][1][0]
    
    # Book-keeping
    batch +=1
    reward_sum = np.sum(reward)/batch_ep
    running_reward = running_reward * 0.99 + reward_sum * 0.01
    print 'batch %d: tot reward: %.0f run mean: %.3f V: %.3f. V-loss: %.3f. pi-loss: %.3f.' %\
    (batch, reward_sum, running_reward, np.mean(value), vloss, piloss)

In [ ]:
np.mean(model.predict(state)[1],0)